# 1. Initialization

In [2]:
import os
import gc
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import xml.dom.minidom as dom
import cx_Oracle as ora
import pyodbc as sql
import re
from datetime import datetime as dt
from sqlalchemy import create_engine

print(f'Pandas version: {pd.__version__}')
print(f'cs-Oracle version: {ora.__version__}')

Pandas version: 1.5.2
cs-Oracle version: 8.3.0


### 1.1 Set Parameters

In [3]:
# Set Parameters

#env = 'QA'
#env = 'STG'
env = 'PRD'

path = 'C:\\Users\\aolguin\\Projects\\CVE\\'
extract_path = path + 'Extracts\\'
xml_path = 'Build\\OI Scripts\\'
mapping_file = 'Build\\Mappings\\Mapping Tables.xlsx'
JDE_files = {'jdepddta': 'Extracts\\jdepddta_f00165_links-2.csv'
            ,'jdelgdta': 'Extracts\\jdelgdta_f00165_links-2.csv'
            }
MAXIMO_files = {'maximo1': 'Extracts\\docinfo_1.csv',
                'maximo2': 'Extracts\\docinfo_2.csv',
                'maximo3': 'Extracts\\docinfo_3.csv'                
               }
#extract_path = 'E:\\eBWeb Test Export\\Projects\\'
#upload_path = '\\\wsidm009pd\\eBHuskyDevice\\Projects\\'

df_extract = {}
df_report = {}
df_JDE_report = {}

target_folder = 'Enterprise:Department Administration:Finance:z-Archives:'
#batch_size = 6000

cnxn_str = {'cdms': 'Driver={SQL Server Native Client 11.0}; Server=csagprd01list; Database=ContentServer; UID=csReadOnly; PWD=Ry62sW781010@@',
#            'cve': 'cs10/cs10tqa55@ORAPRD24:1521/ORAPRD24?encoding=UTF-8&nencoding=UTF-8'
            'cve': 'cs10:cs10tqa55@ORAPRD24:1521/ORAPRD24?encoding=UTF-8&nencoding=UTF-8'
           }

query = {}
query['cdms'] = """
        SELECT  ValStr DATAID_CVE, MAX(Id) DATAID_CDMS
        FROM    LLAttrData
        WHERE   DefId = 253905
        AND     AttrId = 2
        AND     ValStr IS NOT NULL
        AND     ISNUMERIC(ValStr) = 1
        GROUP BY ValStr
        """
query['zmaster'] = """
        SELECT  OldDataID
        FROM    Z_MIGRATION_MASTER
        """
query['cve_root_folders'] = """
        SELECT  DataId, Name
        FROM    dtree
        WHERE   parentid = 2000
        AND     subtype = 0
        ORDER BY Name
        """
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

Oct 13 2023 13:44:16 PM: Done


In [4]:
# Create Database Engines
#ora_engine = create_engine('oracle://' + cnxn_str['cve'], use_nchar_for_unicode=True, coerce_to_unicode=False, text_encoding_errors='replace')
ora_engine = create_engine('oracle://' + cnxn_str['cve'], encoding_errors='replace')
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

Oct 13 2023 13:44:17 PM: Done


### 1.2 Define Functions

In [11]:
def upload_cve_data (dataid: int, save_to_file: bool):
    
    # Build query strings
    query['cve_containers'] = f"""
        WITH    folders AS
                (SELECT  CONNECT_BY_ROOT Name RootFolder
                        ,ParentId
                        ,DataId
                        ,REGEXP_REPLACE(Name, '[^[:print:]]', '') Name
                        ,CASE SubType WHEN 0 THEN 'Folder' WHEN 202 THEN 'Project' WHEN 751 THEN 'Email Folders'   END SubType
                        ,LEVEL FolderLevel
                        ,SYS_CONNECT_BY_PATH(REPLACE(Name, ':', ' '), ':') path
                 FROM   DTree
                 WHERE  SubType IN (0, 202, 751)
                 START WITH DataID = {dataid}
                 CONNECT BY NOCYCLE PRIOR DataID = ParentID)
        SELECT  RootFolder
                ,CASE   WHEN FolderLevel = 2 AND INSTR(path, ':', 1, 3) = 0 THEN SUBSTR(path, INSTR(path, ':', 1, 2)+1)
                        ELSE SUBSTR(path, INSTR(path, ':', 1, 2)+1, INSTR(path, ':', 1, 3)-INSTR(path, ':', 1, 2)-1) END    Level2
                ,CASE   WHEN FolderLevel = 3 AND INSTR(path, ':', 1, 4) = 0 THEN SUBSTR(path, INSTR(path, ':', 1, 3)+1)
                        ELSE SUBSTR(path, INSTR(path, ':', 1, 3)+1, INSTR(path, ':', 1, 4)-INSTR(path, ':', 1, 3)-1) END    Level3
                ,CASE   WHEN FolderLevel = 4 AND INSTR(path, ':', 1, 5) = 0 THEN SUBSTR(path, INSTR(path, ':', 1, 4)+1)
                        ELSE SUBSTR(path, INSTR(path, ':', 1, 4)+1, INSTR(path, ':', 1, 5)-INSTR(path, ':', 1, 4)-1) END    Level4
--                ,CASE   WHEN FolderLevel = 5 AND INSTR(path, ':', 1, 6) = 0 THEN SUBSTR(path, INSTR(path, ':', 1, 5)+1)
--                        ELSE SUBSTR(path, INSTR(path, ':', 1, 5)+1, INSTR(path, ':', 1, 6)-INSTR(path, ':', 1, 5)-1) END    Level5
--                ,FolderLevel
--                ,ParentId
                ,DataId
--                ,Name
--                ,SubType
                ,Path
        FROM    Folders
        """
    query['cve_contents'] = f"""
        SELECT  dt.ParentId
                ,dt.DataId
                ,vd.VersionId
                ,vd.Version
                ,CASE dt.SubType WHEN 1 THEN 'Shortcut'              WHEN 140 THEN 'Url'
                                 WHEN 136 THEN 'Compound Document'   WHEN 144 THEN 'Document'
                                 WHEN 557 THEN 'Compound Email'      WHEN 749 THEN 'Email'
                                 WHEN 411 THEN 'Physical Item'
                                 WHEN 0 THEN 'Folder'                WHEN 202 THEN 'Project' 
                                 WHEN 751 THEN 'Email Folders'                                  END SubType
                ,vd.DataSize
        FROM    DTree dt
        LEFT OUTER JOIN DVersData vd ON vd.DocId = dt.DataId
        WHERE   dt.SubType IN (1, 136, 140, 144, 411, 557, 749, 0, 202, 751)
        AND     dt.Deleted = 0
        START WITH dt.DataID = {dataid}
        CONNECT BY NOCYCLE PRIOR dt.DataID = dt.ParentID
        """ 
    
    # Upload Data from Database
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loading CVE Folder and Document data...')
#    cnxn = ora.connect(cnxn_str['cve'])
    cnxn = ora_engine.connect()
    for i in ['cve_containers', 'cve_contents']:
        df_extract[i] = pd.read_sql(query[i], cnxn)
        print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Loaded {i}= {df_extract[i].shape[0]} rows')
        if save_to_file:
            df_extract[i].to_csv(extract_path + i + '_' + str(dataid) + '.csv')
            print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Saving to csv {i}= {df_extract[i].shape[0]} rows')
            release_extract_memory([i])
    cnxn.close()

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

Sep 25 2023 14:25:31 PM: Done


In [5]:
def upload_cve_cats (dataid: int):
    
    # Build query strings
    query['cve_categories'] = f"""
        WITH documents AS
                (SELECT  DataId
                        ,VersionNum
                 FROM 	DTree dt
                 WHERE 	SubType IN (1, 136, 140, 144, 411, 557, 749)
                 START WITH DataID = {dataid}
                 CONNECT BY NOCYCLE PRIOR DataID = ParentID)
        SELECT  ValStr
                ,COUNT(DISTINCT DataId) items
        FROM    llAttrData att
        JOIN    documents do ON do.DataId = att.Id AND do.VersionNum = att.Vernum
        WHERE   AttrId = 1
        GROUP BY ValStr
        ORDER BY ValStr
        """
    
    # Upload Data from Database
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loading CVE Folder and Document data...')
#    cnxn = ora.connect(cnxn_str['cve'])
    cnxn = ora_engine.connect()
    df_extract['cve_categories'] = pd.read_sql(query['cve_categories'], cnxn)
    cnxn.close()
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Loaded categories = {df_extract["cve_categories"].shape[0]} rows')

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')




Oct 13 2023 14:20:05 PM: Done


In [20]:
def create_data_assessment_report(root_folder:str):

    # 1. Merge with CDMS, JDE, MAXIMO TABLES

    # Merge CDMS
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Merging CDMS...')
    if 'DATAID_CVE' in df_extract['cve_contents'].columns: df_extract['cve_contents'].drop(['DATAID_CVE'], axis=1, inplace=True)
    if 'DATAID_CDMS' in df_extract['cve_contents'].columns: df_extract['cve_contents'].drop(['DATAID_CDMS'], axis=1, inplace=True)
    df_extract['cve_contents'] = df_extract['cve_contents'].merge(df_extract['CDMS'], how='left', left_on='dataid', right_on= 'DATAID_CVE')
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Merged CDMS = {df_extract["cve_contents"].shape[0]} rows')

    # Merge ZMASTER
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Merging ZMASTER...')
    if 'DATAID_ZMASTER' in df_extract['cve_contents'].columns: df_extract['cve_contents'].drop(['DATAID_ZMASTER'], axis=1, inplace=True)
    df_extract['cve_contents'] = df_extract['cve_contents'].merge(df_extract['ZMASTER'], how='left', left_on='dataid', right_on= 'DATAID_ZMASTER')
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Merged ZMASTER = {df_extract["cve_contents"].shape[0]} rows')

    # Merge JDE
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Merging JDE...')
    if 'DATAID_JDE' in df_extract['cve_contents'].columns: df_extract['cve_contents'].drop(['DATAID_JDE'], axis=1, inplace=True)
    df_extract['cve_contents'] = df_extract['cve_contents'].merge(df_extract['JDE'], how='left', left_on='dataid', right_on= 'DATAID_JDE')
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Merged JDE = {df_extract["cve_contents"].shape[0]} rows')

    # Merge MAXIMO
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Merging MAXIMO...')
    if 'DATAID_MAX' in df_extract['cve_contents'].columns: df_extract['cve_contents'].drop(['DATAID_MAX'], axis=1, inplace=True)
    df_extract['cve_contents'] = df_extract['cve_contents'].merge(df_extract['MAXIMO'], how='left', left_on='dataid', right_on= 'DATAID_MAX')
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Merged MAXIMO = {df_extract["cve_contents"].shape[0]} rows')


    # 2. Generate Summary Matrix per PARENTID
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Grouping per ParentID...')

    # Group CONTENTS by PARENTID
    df_extract['cve_contents'].sort_values(by=['parentid', 'subtype'], inplace=True, na_position='first')
    df_content_sum = df_extract['cve_contents'][['parentid', 'subtype', 'dataid', 'versionid', 'datasize', 'DATAID_ZMASTER', 'DATAID_CDMS', 'DATAID_JDE', 'DATAID_MAX']].groupby(by=['parentid', 'subtype'], dropna=False).agg({'dataid': pd.Series.nunique, 'versionid': pd.Series.count, 'datasize': lambda x: np.sum(x)/1024/1024/1024, 'DATAID_ZMASTER': pd.Series.nunique, 'DATAID_CDMS': pd.Series.nunique, 'DATAID_JDE': pd.Series.nunique, 'DATAID_MAX': pd.Series.nunique})
    df_content_sum.reset_index(inplace=True)
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Grouped cve_contents = {df_content_sum.shape[0]} rows')

    # Transverse the grouped data
    df_content_matrix = df_content_sum[['parentid']].drop_duplicates()
    subtype_list = df_content_sum['subtype'].unique()
    subtype_list.sort()
    for subtype in subtype_list:
        if subtype == 'Document':
            df = df_content_sum[['parentid', 'dataid', 'DATAID_ZMASTER', 'DATAID_CDMS', 'DATAID_JDE', 'DATAID_MAX', 'versionid', 'datasize']][(df_content_sum['subtype']) == subtype]
            df.rename({'dataid': subtype, 'DATAID_ZMASTER': subtype+'_ZTAB', 'DATAID_CDMS': subtype+'_CDMS', 'DATAID_JDE': subtype+'_JDE', 'DATAID_MAX': subtype+'_MAX', 'versionid': subtype+'_VERS', 'datasize': subtype+'_GB'}, axis=1, inplace=True)
        else:
            df = df_content_sum[['parentid', 'dataid', 'DATAID_ZMASTER', 'DATAID_CDMS', 'DATAID_JDE', 'DATAID_MAX']][(df_content_sum['subtype']) == subtype]
            df.rename({'dataid': subtype, 'DATAID_ZMASTER': subtype+'_ZTAB', 'DATAID_CDMS': subtype+'_CDMS', 'DATAID_JDE': subtype+'_JDE', 'DATAID_MAX': subtype+'_MAX'}, axis=1, inplace=True)
        df_content_matrix = df_content_matrix.merge(df, how='left', on='parentid')
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Transversed grouped content = {df_content_matrix.shape[0]} rows')


    # 3. Generate final report per Root Folder
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Generating Final Report...')

    # Merge CONTENTS with CONTAINERS
    df_extract['cve_containers'] = df_extract['cve_containers'].merge(df_content_matrix, how='left', left_on='dataid', right_on='parentid', suffixes=('_FOL', '_DOC'))
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Merged Content & Containers = {df_extract["cve_containers"].shape[0]} rows')

    # Roll-up to Level 4
    df_extract['cve_containers'].sort_values(by='path', inplace=True, na_position='first')
    df_extract['cve_containers'].drop(['dataid', 'path', 'parentid'], axis=1, inplace=True)
    df_report[root_folder] = df_extract['cve_containers'].groupby(by=['rootfolder', 'level2', 'level3', 'level4'], dropna=False).sum()
    df_report[root_folder].sort_values(by=['rootfolder', 'level2', 'level3', 'level4'], inplace=True, na_position='first')
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Rolled-up to Level 4 = {df_report[root_folder].shape[0]} rows')

    # Report output
    df_report[root_folder].to_excel(path + f'CVE - Data Analysis - {root_folder}.xlsx')
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Report Output = {df_report[root_folder].shape[0]} rows')

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

Sep 25 2023 15:04:32 PM: Done


In [ ]:
#def build_create_oi(df: pd.DataFrame, type: str, output_file: str, sort_list: list, legacy_id: str, category_name: str):
def build_create_oi(df: pd.DataFrame, type: str, output_file: str, sort_list: list, legacy_id: str, category_name: str, cat_atts: list):
    for batch in df['batch'][(df['batch'].notnull())].sort_values().unique():
        LegacyID_ant = ''
        xml_file_name = 'create-' + type + '-' + output_file + '-' + str(batch).zfill(2) + '.xml'
        df_batch = df[(df['batch'] == batch)].sort_values(by=sort_list)
        root = ET.Element('import')
        tree = ET.ElementTree(root)
        for key, value in df_batch.iterrows():
            node = ET.Element('node')
            node.set('type', type)
            location = ET.SubElement(node, 'location')
            location.text = value['location']
            title = ET.SubElement(node, 'title')
            title.text = value['title']
            if type in ['document', 'email', 'caddocument']:
                ET.SubElement(node, 'description').text = value['description']
                ET.SubElement(node, 'created').text = value['created']
                ET.SubElement(node, 'createdby').text = value['createdby']
                ET.SubElement(node, 'modified').text = value['modified']
                ET.SubElement(node, 'file').text = value['file']
                ET.SubElement(node, 'filename').text = value['filename']
                ET.SubElement(node, 'mime').text = value['mime']                    
                if value[legacy_id] == LegacyID_ant:
                    node.set('action', 'addversion')
                    node.remove(location)
                    node.remove(title)
                    ET.SubElement(node, 'location').text = value['location'] + ':' + value['title']
                else:
                    node.set('action', 'create')
                LegacyID_ant = value[legacy_id]
            if type == 'url':
                node.set('action', 'create')
                ET.SubElement(node, 'url').text = value['url']
            if type == 'folder':
                node.set('action', 'create')
            if type == 'alias':
                node.set('action', 'create')
                ET.SubElement(node, 'alias').text = value['alias']
            if type == 'compounddoc':
                node.set('action', 'create')
            if len(cat_atts) != 0 and category_name != '':
                category = ET.Element('category')
                category.set('name', category_name)
                for att in cat_atts:
                    if len(value[att]) > 0: ET.SubElement(category, 'attribute', attrib={'name': att}).text = value[att]
                node.append(category)
            root.append(node)
        xml = ET.tostring(root, encoding='utf-8', method='xml')
        xml_parsed = dom.parseString(xml).toprettyxml()
        xml_file = open(path + xml_path + xml_file_name, 'w',  encoding='utf-8')
        xml_file.write(xml_parsed)
        xml_file.close()
        print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     {xml_file_name} = {len(root)} nodes out of {df_batch.shape[0]}')
        root.clear()
        
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

In [ ]:
def build_update_cats_oi(df: pd.DataFrame, type: str, output_file: str, sort_list: list, category_name: str, cat_atts: list):
    for batch in df['batch'][(df['batch'].notnull())].sort_values().unique():
        LegacyID_ant = ''
        xml_file_name = 'update-' + type + '-' + output_file + '-' + str(batch).zfill(2) + '.xml'
        df_batch = df[(df['batch'] == batch)].sort_values(by=sort_list)
        root = ET.Element('import')
        tree = ET.ElementTree(root)
        for key, value in df_batch.iterrows():
            node = ET.Element('node')
            node.set('type', type)
            node.set('action', 'update')
            ET.SubElement(node, 'location').text = value['location'] + ':' + value['title']
#            ET.SubElement(node, 'location').text = value['location']
#            ET.SubElement(node, 'description').text = value['description']
            if len(cat_atts) != 0 and category_name != '':
                category = ET.Element('category')
                category.set('name', category_name)
                for att in cat_atts:
                    if len(value[att]) > 0:
#                        print(f'{value[att]} {len(value[att])}')
                        ET.SubElement(category, 'attribute', attrib={'name': att}).text = value[att]
                node.append(category)
            root.append(node)
        xml = ET.tostring(root, encoding='utf-8', method='xml')
        xml_parsed = dom.parseString(xml).toprettyxml()
        xml_file = open(path + xml_path + xml_file_name, 'w',  encoding='utf-8')
        xml_file.write(xml_parsed)
        xml_file.close()
        print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     {xml_file_name} = {len(root)} nodes out of {df_batch.shape[0]}')
        root.clear()
        
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

In [ ]:
def build_delete_oi(df: pd.DataFrame, type: str, output_file: str, sort_list: list):
    for batch in df['batch'][(df['batch'].notnull())].sort_values().unique():
        xml_file_name = 'delete-' + type + '-' + output_file + '-' + str(batch).zfill(2) + '.xml'
        df_batch = df[(df['batch'] == batch)].sort_values(by=sort_list, ascending=False)
        root = ET.Element('import')
        tree = ET.ElementTree(root)
        for key, value in df_batch.iterrows():
            node = ET.Element('node')
            node.set('type', type)
            node.set('action', 'delete')
            ET.SubElement(node, 'location').text = value['location'] + ':' + value['title']
#            ET.SubElement(node, 'title').text = value['title']
            root.append(node)
        xml = ET.tostring(root, encoding='utf-8', method='xml')
        xml_parsed = dom.parseString(xml).toprettyxml()
        xml_file = open(path + xml_path + xml_file_name, 'w',  encoding='utf-8')
        xml_file.write(xml_parsed)
        xml_file.close()
        print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     {xml_file_name} = {len(root)} nodes out of {df_batch.shape[0]}')
        root.clear()
        
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

In [4]:
def release_extract_memory(names:list):
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Releasing Memory...')
    
    total_memory_used = 0
    memory_freed = 0
    for name in names:
        # Calculate used memory
        total_memory_used =+ df_extract[name].memory_usage().sum()
    total_memory_used = total_memory_used/1024/1024
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Memory Usage = {total_memory_used} MB')

    # Delete DataFrames
    for name in names:
        del df_extract[name]
        df_extract[name] = pd.DataFrame()
        memory_freed =+ gc.collect()
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Memory Freed = {memory_freed}')

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

Sep 25 2023 13:57:13 PM: Done


# 2. Prepare Data

### 2.1 Prepare & Analyze JDE Data

In [ ]:
# JDE Files Analysis

for file in JDE_files.keys():
    # Upload JDE Data
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loading JDE data...')
    df_extract[file] = pd.read_csv(path + JDE_files[file])
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loaded JDE {file} = {df_extract[file].shape[0]} rows')

    # Calculate Server link
    df_extract[file]['server_link'] = df_extract[file].apply(lambda x: x['url'][0:x['url'].find('/', 8)], axis=1)
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Calculated Server Link {file} = {df_extract[file].shape[0]} rows')

    # Extract DataIds List
    # http://livelink, http://livelink.encana.com, https://contentserver.cenovus.com, http://doc, http://doc.encana.com
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Extracting candidate DataIds...')
    df_extract[file]['cve_dataid_list'] = df_extract[file].apply(lambda x: None if x['server_link'] != 'http://livelink' and x['server_link'] != 'http://livelink.encana.com' and x['server_link'] != 'https://contentserver.cenovus.com' and x['server_link'] != 'http://doc' and x['server_link'] != 'http://doc.encana.com'
                                                    else (re.findall('(?<=docref=)[0-9]*', x['url']) if x['url'].find('docref=') != -1 
                                                          else (re.findall('(?<=docid=)[0-9]*', x['url']) if x['url'].find('docid=') != -1
                                                                else (re.findall('(?<=nodeid=)[0-9]*', x['url']) if x['url'].find('nodeid=') != -1
                                                                      else (re.findall('(?<=nodeId=)[0-9]*', x['url']) if x['url'].find('nodeId=') != -1
                                                                            else (re.findall('(?<=\/open/)[0-9]*', x['url']) if x['url'].find('/open/') != -1
                                                                                  else (re.findall('(?<=\/Open/)[0-9]*', x['url']) if x['url'].find('/Open/') != -1
                                                                                        else (re.findall('(?<=objId=)[0-9]*', x['url']) if x['url'].find('objId=') != -1
                                                                                              else (re.findall('(?<=objid=)[0-9]*', x['url']) if x['url'].find('objid=') != -1
                                                                                                    else (re.findall('(?<=\/nodes/)[0-9]*', x['url']) if x['url'].find('/nodes/') != -1
                                                                                                          else 'no_value_found'))))))))), axis=1)
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Extracted DataIds for {file} = {df_extract[file].shape[0]} rows')

    # Extract DataIds
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Converting candidate DataIds to numeric type...')
    df_extract[file]['cve_dataid'] = df_extract[file][['cve_dataid_list', 'server_link']].apply(lambda x: None if x['server_link'] != 'http://livelink' and x['server_link'] != 'http://livelink.encana.com' and x['server_link'] != 'https://contentserver.cenovus.com' and x['server_link'] != 'http://doc' and x['server_link'] != 'http://doc.encana.com' 
                                                                                                else (0 if len(x['cve_dataid_list']) == 0
                                                                                                      else (x['cve_dataid_list'][0] if x['cve_dataid_list'] != 'no_value_found'
                                                                                                            else x['cve_dataid_list'])), axis=1)
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Converted DataIds for {file} = {df_extract[file].shape[0]} rows')

    # Calculate Type
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Calculating Type...')
    df_extract[file]['type'] = df_extract[file][['cve_dataid_list', 'server_link']].apply(lambda x: 'not cs' if x['server_link'] != 'http://livelink' and x['server_link'] != 'http://livelink.encana.com' and x['server_link'] != 'https://contentserver.cenovus.com' and x['server_link'] != 'http://doc' and x['server_link'] != 'http://doc.encana.com' 
                                                                                          else ('no_value' if len(x['cve_dataid_list']) == 0
                                                                                                else ('candidate_value' if x['cve_dataid_list'] != 'no_value_found'
                                                                                                      else x['cve_dataid_list'])), axis=1)
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Calculated Type for {file} = {df_extract[file].shape[0]} rows')

    # Remove cve_dataid_list
    if 'cve_dataid_list' in df_extract[file].columns: df_extract[file].drop(['cve_dataid_list'], axis=1, inplace=True)

    # Reports for JDE links
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Creating JDE extracts...')
    # Counts
    df_extract[file].sort_values(by=['server_link', 'url'], inplace=True)
    df_extract[file].groupby(by=['server_link', 'type']).agg({'url': [pd.Series.count, pd.Series.nunique]}).to_excel(path + f'CVE - Data Analysis - JDE - {file} Summary.xlsx')
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Count Links per Server {file} = {df_extract[file].shape[0]} rows')
    # All links Translation
    df_extract[file].to_csv(extract_path + f'CVE - Data Analysis - JDE - {file} Translated links.csv')
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Saving Links translation {file} = {df_extract[file].shape[0]} rows')
    # Scorpio and ECN slices
    df_extract[file][(df_extract[file]['server_link']) == 'http://scorpio'].to_csv(extract_path + f'CVE - Data Analysis - JDE - {file} Scorpio links.csv', index=False)
    df_extract[file][(df_extract[file]['server_link']) == 'http://ecn.encana.com'].to_csv(extract_path + f'CVE - Data Analysis - JDE - {file} Ecn links.csv', index=False)
    # Save DataIds to file
    pd.DataFrame(df_extract[file]['cve_dataid'][(df_extract[file]['type'] == 'candidate_value')].unique(), columns=['DATAID_JDE']).to_csv(extract_path + f'DataIds_JDE_{file}.csv', index=False)
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Saved DataIds to file {file} = {df_extract[file].shape[0]} rows')
    
    # Release Memory
    release_extract_memory([file])
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Memory released {file} = {df_extract[file].shape[0]} rows')

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

### 2.2 Prepare and Analyze MAXIMO Data

In [ ]:
# MAXIMO Files Analysis

# Upload MAXIMO Data
df_extract['maximo'] = pd.DataFrame()
for file in MAXIMO_files.keys():
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loading MAXIMO data...')
    df_extract[file] = pd.read_csv(path + MAXIMO_files[file], encoding_errors='replace')
    if df_extract['maximo'].empty:
        df_extract['maximo'] = df_extract[file]
    else:
        df_extract['maximo'] = pd.concat([df_extract['maximo'], df_extract[file]])
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loaded {file} = {df_extract[file].shape[0]} rows')
df_extract['maximo'].rename(columns={'URLNAME':'url'}, inplace=True)
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loaded all maximo files = {df_extract["maximo"].shape[0]} rows')


file = 'maximo'
# Calculate Server link
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Calculating server link...')
df_extract[file]['server_link'] = df_extract[file].apply(lambda x: x['url'][0:x['url'].find('/', 8)], axis=1)
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Calculated Server Link {file} = {df_extract[file].shape[0]} rows')

# Extract DataIds List
# http://livelink, http://livelink.encana.com, https://contentserver.cenovus.com, http://doc, http://doc.encana.com
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Extracting candidate DataIds...')
df_extract[file]['cve_dataid_list'] = df_extract[file].apply(lambda x: None if x['server_link'].find('livelink') == -1 and x['server_link'].find('contentserver') == -1 and x['server_link'].find('http://doc') == -1
                                                else (re.findall('(?<=docref=)[0-9]*', x['url']) if x['url'].find('docref=') != -1 
                                                      else (re.findall('(?<=docid=)[0-9]*', x['url']) if x['url'].find('docid=') != -1
                                                            else (re.findall('(?<=nodeid=)[0-9]*', x['url']) if x['url'].find('nodeid=') != -1
                                                                  else (re.findall('(?<=nodeId=)[0-9]*', x['url']) if x['url'].find('nodeId=') != -1
                                                                        else (re.findall('(?<=\/open/)[0-9]*', x['url']) if x['url'].find('/open/') != -1
                                                                              else (re.findall('(?<=\/Open/)[0-9]*', x['url']) if x['url'].find('/Open/') != -1
                                                                                    else (re.findall('(?<=objId=)[0-9]*', x['url']) if x['url'].find('objId=') != -1
                                                                                          else (re.findall('(?<=objid=)[0-9]*', x['url']) if x['url'].find('objid=') != -1
                                                                                                else (re.findall('(?<=\/nodes/)[0-9]*', x['url']) if x['url'].find('/nodes/') != -1
                                                                                                      else 'no_value_found'))))))))), axis=1)
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Extracted DataIds for {file} = {df_extract[file].shape[0]} rows')

# Extract DataIds
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Converting candidate DataIds to numeric type...')
df_extract[file]['cve_dataid'] = df_extract[file][['cve_dataid_list', 'server_link']].apply(lambda x: None if x['server_link'].find('livelink') == -1 and x['server_link'].find('contentserver') == -1 and x['server_link'].find('http://doc') == -1
                                                                                            else (0 if len(x['cve_dataid_list']) == 0
                                                                                                  else (x['cve_dataid_list'][0] if x['cve_dataid_list'] != 'no_value_found'
                                                                                                        else x['cve_dataid_list'])), axis=1)
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Converted DataIds for {file} = {df_extract[file].shape[0]} rows')

# Calculate Type
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Calculating Type...')
df_extract[file]['type'] = df_extract[file][['cve_dataid_list', 'server_link']].apply(lambda x: 'not cs' if x['server_link'].find('livelink') == -1 and x['server_link'].find('contentserver') == -1 and x['server_link'].find('http://doc') == -1
                                                                                      else ('no_value' if len(x['cve_dataid_list']) == 0
                                                                                            else ('candidate_value' if x['cve_dataid_list'] != 'no_value_found'
                                                                                                  else x['cve_dataid_list'])), axis=1)
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Calculated Type for {file} = {df_extract[file].shape[0]} rows')

# Remove cve_dataid_list
if 'cve_dataid_list' in df_extract[file].columns: df_extract[file].drop(['cve_dataid_list'], axis=1, inplace=True)

# Reports for MAXIMO links
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Creating MAXIMO extracts...')
# Counts
df_extract[file].sort_values(by=['server_link', 'url'], inplace=True)
df_extract[file].groupby(by=['server_link', 'type']).agg({'url': [pd.Series.count, pd.Series.nunique]}).to_excel(path + f'CVE - Data Analysis - MAX - {file} Summary.xlsx')
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Count Links per Server {file} = {df_extract[file].shape[0]} rows')
# All links Translation
df_extract[file].to_csv(extract_path + f'CVE - Data Analysis - MAX - {file} Translated links.csv')
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Saving Links translation {file} = {df_extract[file].shape[0]} rows')
# Save DataIds to file
pd.DataFrame(df_extract[file]['cve_dataid'][(df_extract[file]['type'] == 'candidate_value')].unique(), columns=['DATAID_MAX']).to_csv(extract_path + f'DataIds_MAX_{file}.csv', index=False)
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Saved DataIds to file {file} = {df_extract[file].shape[0]} rows')

# Release Memory
release_extract_memory([file])
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Memory released {file} = {df_extract[file].shape[0]} rows')

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

### 2.3 Prepare CDMS Data 

In [ ]:
# Upload CDMS Data
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loading CDMS data...')
cnxn = sql.connect(cnxn_str['cdms'])
df_extract['cdms'] = pd.read_sql(query['cdms'], cnxn)
cnxn.close()
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loaded CDMS= {df_extract["cdms"].shape[0]} rows')

# Change DataTypes to match merge 
#df_extract['cdms'] = df_extract['cdms'].astype({'DATAID_CVE': 'int64'}, copy='False')

# Save DataIds to file
df_extract['cdms'].to_csv(extract_path + f'DataIds_CDMS.csv', index=False)
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Saved DataIds to file = {df_extract["cdms"].shape[0]} rows')

# Release Memory
release_extract_memory(['cdms'])
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Memory released = {df_extract["cdms"].shape[0]} rows')

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

### 2.4 Prepare Z_MIGRATION_MASTER Data

In [ ]:
# Upload Z_MIGRATION_MASTER Data
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loading Z_MIGRATION_MASTER data...')
cnxn = ora_engine.connect()
df_extract['zmaster'] = pd.read_sql(query['zmaster'], cnxn)
cnxn.close()
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loaded Z_MIGRATION_MASTER = {df_extract["zmaster"].shape[0]} rows')

# Change DataTypes to match merge 
#df_extract['cdms'] = df_extract['cdms'].astype({'DATAID_CVE': 'int64'}, copy='False')

# Save DataIds to file
df_extract['zmaster'].drop_duplicates(inplace=True)
df_extract['zmaster'].rename(columns={'olddataid': 'DATAID_ZMASTER'}, inplace=True)
df_extract['zmaster'].to_csv(extract_path + f'DataIds_ZMASTER.csv', index=False)
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Saved DataIds to file = {df_extract["zmaster"].shape[0]} rows')

# Release Memory
release_extract_memory(['zmaster'])
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Memory released = {df_extract["zmaster"].shape[0]} rows')

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

# 3. Data Assessment & Reports

### 3.1 Upload CDMS, JDE, Maximo DataIds flat files

In [12]:
# Upload JDE DataIds from file
df_extract['JDE'] = pd.concat([pd.read_csv(extract_path + 'DataIds_JDE_jdepddta.csv'), pd.read_csv(extract_path + 'DataIds_JDE_jdelgdta.csv')]).drop_duplicates()
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Uploaded JDE DataIds = {df_extract["JDE"].shape[0]} rows')

# Upload MAXIMO DataIds from file
df_extract['MAXIMO'] = pd.read_csv(extract_path + 'DataIds_MAX_maximo.csv').drop_duplicates()
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Uploaded MAXIMO DataIds = {df_extract["MAXIMO"].shape[0]} rows')

# Upload CDMS DataIds from file
df_extract['CDMS'] = pd.read_csv(extract_path + 'DataIds_CDMS.csv').drop_duplicates()
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Uploaded CDMS DataIds = {df_extract["CDMS"].shape[0]} rows')

# Upload ZMASTER DataIds from file
df_extract['ZMASTER'] = pd.read_csv(extract_path + 'DataIds_ZMASTER.csv').drop_duplicates()
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Uploaded Z_MIGRATION_MASTER DataIds = {df_extract["ZMASTER"].shape[0]} rows')

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

Sep 25 2023 14:26:12 PM: Uploaded JDE DataIds = 14201637 rows
Sep 25 2023 14:26:13 PM: Uploaded MAXIMO DataIds = 979353 rows
Sep 25 2023 14:26:56 PM: Uploaded CDMS DataIds = 19146903 rows
Sep 25 2023 14:27:14 PM: Uploaded Z_MIGRATION_MASTER DataIds = 16348452 rows
Sep 25 2023 14:27:14 PM: Done


### 3.2 Upload Folders & Contents Data and create Reports per folder

In [40]:
# ***** ALL Root Folders except Upstream Operations *****

#(7 hrs 20 min)
# Create list of extract Folders: Root Folders
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loading root folders...')
cnxn = ora_engine.connect()
df_extract['cve_folders'] = pd.read_sql(query['cve_root_folders'], cnxn)
cnxn.close()
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Loaded cve_root_folders = {df_extract["cve_folders"].shape[0]} rows')

#Remove Upstream Operations
df_extract['cve_folders'].drop(axis=0, index=[30], inplace=True)

# Build Data Assessment Report
#for idx, row in df_extract['cve_folders'][(df_extract['cve_folders'].index == 0)].iterrows():
for idx, row in df_extract['cve_folders'].iterrows():
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: ********** FOLDER = {row["name"]} **********')
    # Upload container and contents for root folder from Database
    upload_cve_data(row['dataid'], save_to_file=False)
    
    # Merge all data and create report
    create_data_assessment_report(row['name'])

    # Release Memory
    release_extract_memory(['cve_containers', 'cve_contents'])


Sep 26 2023 06:08:37 AM: Loading root folders...
Sep 26 2023 06:08:37 AM:     Loaded cve_root_folders = 32 rows
Sep 26 2023 06:08:37 AM: ********** FOLDER = Application Data **********
Sep 26 2023 06:08:37 AM: Loading CVE Folder and Document data...
Sep 26 2023 06:14:31 AM:     Loaded cve_containers= 513953 rows
Sep 26 2023 06:30:05 AM:     Loaded cve_contents= 2746760 rows
Sep 26 2023 06:30:05 AM: Merging CDMS...
Sep 26 2023 06:30:29 AM:     Merged CDMS = 2746760 rows
Sep 26 2023 06:30:29 AM: Merging ZMASTER...
Sep 26 2023 06:30:56 AM:     Merged ZMASTER = 2746760 rows
Sep 26 2023 06:30:56 AM: Merging JDE...
Sep 26 2023 06:31:33 AM:     Merged JDE = 2746760 rows
Sep 26 2023 06:31:33 AM: Merging MAXIMO...
Sep 26 2023 06:31:38 AM:     Merged MAXIMO = 2746760 rows
Sep 26 2023 06:31:38 AM: Grouping per ParentID...
Sep 26 2023 06:45:48 AM:     Grouped cve_contents = 512741 rows
Sep 26 2023 06:45:51 AM:     Transversed grouped content = 512737 rows
Sep 26 2023 06:45:51 AM: Generating Final 

In [38]:
# ***** Upstream Operations *****

#(7 hrs)
# Create list of extract Folders: Upstream Operations Subfolders
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loading CVE subfolders...')
query['cve_folders'] = """
        SELECT  DataId, Name
        FROM    dtree
        WHERE   parentid = 4673
        AND     subtype = 0
        ORDER BY Name
        """
cnxn = ora_engine.connect()
df_extract['cve_folders'] = pd.read_sql(query['cve_folders'], cnxn)
cnxn.close()
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Loaded cve_folders = {df_extract["cve_folders"].shape[0]} rows')

# Build Data Assessment Report
#for idx, row in df_extract['cve_folders'][(df_extract['cve_folders'].index == 31)].iterrows():
for idx, row in df_extract['cve_folders'].iterrows():
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: ********** FOLDER = {row["name"]} **********')
    # Upload container and contents for root folder from Database
    upload_cve_data(row['dataid'], save_to_file=False)
    
    # Merge all data and create report
    create_data_assessment_report(row['name'])

    # Release Memory
    release_extract_memory(['cve_containers', 'cve_contents'])

# Consolidate Upstream Operations subfolders
#Rollup subfolders to Level 3
for idx, row in df_extract['cve_folders'].iterrows():
    root_folder = row['name']
    df_report[root_folder].sort_values(by=['rootfolder', 'level2', 'level3', 'level4'], inplace=True, na_position='first')
    df_report[root_folder + '_l3'] = df_report[root_folder].groupby(by=['rootfolder', 'level2', 'level3'], dropna=False).sum()
    df_report[root_folder + '_l3'].sort_values(by=['rootfolder', 'level2', 'level3'], inplace=True, na_position='first')
    df_report[root_folder + '_l3'].to_excel(path + f'CVE - Data Analysis - {root_folder + "_l3"}.xlsx')
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Rolled-up to Level 3 = {df_report[root_folder + "_l3"].shape[0]} rows')
#Concatenate all level 3 subfolders
root_folder = 'Upstream Operations'
df_report[root_folder] = pd.concat([df_report['Canadian Plains_NOMIGRATE_l3'],
                                    df_report['Christina Lake_NOMIGRATE_l3'],
                                    df_report['Cross-Region_NOMIGRATE'],
                                    df_report['Finance_l3'],
                                    df_report['Foothills Region_NOMIGRATE_l3'],
                                    df_report['Foster Creek_l3'],
                                    df_report['Upstream Business Services_l3']])

# Report output
df_report[root_folder].to_excel(path + f'CVE - Data Analysis - {root_folder}.xlsx')
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Report Output = {df_report[root_folder].shape[0]} rows')


Sep 23 2023 06:14:31 AM: ********** FOLDER = Canadian Plains_NOMIGRATE **********
Sep 23 2023 06:14:31 AM: Loading CVE Folder and Document data...
Sep 23 2023 06:15:22 AM:     Loaded cve_containers= 23282 rows
Sep 23 2023 06:16:46 AM:     Loaded cve_contents= 293143 rows
Sep 23 2023 06:16:46 AM: Merging CDMS...
Sep 23 2023 06:17:09 AM:     Merged CDMS = 293143 rows
Sep 23 2023 06:17:09 AM: Merging ZMASTER...
Sep 23 2023 06:17:28 AM:     Merged ZMASTER = 293143 rows
Sep 23 2023 06:17:28 AM: Merging JDE...
Sep 23 2023 06:18:05 AM:     Merged JDE = 293143 rows
Sep 23 2023 06:18:05 AM: Merging MAXIMO...
Sep 23 2023 06:18:07 AM:     Merged MAXIMO = 293143 rows
Sep 23 2023 06:18:07 AM: Grouping per ParentID...
Sep 23 2023 06:18:59 AM:     Grouped cve_contents = 33008 rows
Sep 23 2023 06:19:00 AM:     Transversed grouped content = 29722 rows
Sep 23 2023 06:19:00 AM: Generating Final Report...
Sep 23 2023 06:19:00 AM:     Merged Content & Containers = 23282 rows
Sep 23 2023 06:19:00 AM:     Ro

Sep 23 2023 13:18:35 PM:     Merged Content & Containers = 839941 rows
Sep 23 2023 13:18:40 PM:     Rolled-up to Level 4 = 850 rows
Sep 23 2023 13:18:42 PM:     Report Output = 850 rows
Sep 23 2023 13:18:42 PM: Releasing Memory...
Sep 23 2023 13:18:42 PM:     Memory Usage = 531.0612487792969 MB
Sep 23 2023 13:18:44 PM:     Memory Freed = 0


In [ ]:
# Examples to roll-up to higher folder level

# Roll-up to Root Level
root_folder = 'Oil Sands'
df_report[root_folder].sort_values(by=['rootfolder', 'level2', 'level3', 'level4'], inplace=True, na_position='first')
df_report[root_folder + '_l1'] = df_report[root_folder].groupby(by=['rootfolder'], dropna=False).sum()
df_report[root_folder + '_l1'].sort_values(by=['rootfolder'], inplace=True, na_position='first')
df_report[root_folder + '_l1'].to_excel(path + f'CVE - Data Analysis - {root_folder + "_l1"}.xlsx')
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Rolled-up to Level 1 = {df_report[root_folder + "_l1"].shape[0]} rows')

#df_report[root_folder+'_l1'] = df_extract['cve_containers'].groupby(by=['rootfolder'], dropna=False).sum()
#print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Rolled-up to Level 1 = {df_report[root_folder].shape[0]} rows')

# Roll-up to Level 2
for idx, row in df_extract['cve_folders'].iterrows():
    root_folder = row['name']
    df_report[root_folder].sort_values(by=['rootfolder', 'level2', 'level3', 'level4'], inplace=True, na_position='first')
    df_report[root_folder + '_l2'] = df_report[root_folder].groupby(by=['rootfolder', 'level2'], dropna=False).sum()
    df_report[root_folder + '_l2'].sort_values(by=['rootfolder', 'level2'], inplace=True, na_position='first')
    df_report[root_folder + '_l2'].to_excel(path + f'CVE - Data Analysis - {root_folder + "_l2"}.xlsx')
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Rolled-up to Level 2 = {df_report[root_folder + "_l2"].shape[0]} rows')

# Roll-up to Level 3
root_folder = 'Corporate Finance'
df_report[root_folder].sort_values(by=['rootfolder', 'level2', 'level3', 'level4'], inplace=True, na_position='first')
df_report[root_folder + '_l3'] = df_report[root_folder].groupby(by=['rootfolder', 'level2', 'level3'], dropna=False).sum()
df_report[root_folder + '_l3'].sort_values(by=['rootfolder', 'level2', 'level3'], inplace=True, na_position='first')
df_report[root_folder + '_l3'].to_excel(path + f'CVE - Data Analysis - {root_folder + "_l3"}.xlsx')
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Rolled-up to Level 3 = {df_report[root_folder + "_l3"].shape[0]} rows')

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

### 3.3 Create consolidated  reports for JDE and MAXIMO

In [31]:
# Reset index for everything, so values are split in columns instead of concatenated in one column in report
for folder in df_report.keys():
    df_report[folder].reset_index(inplace=True)


In [37]:
# Load Upstream Operations after being edited manually
df_report['Upstream Operations'] = pd.read_excel(extract_path + 'CVE - Data Analysis - Upstream Operations-compiled.xlsx')
df_report['Upstream Operations']

,rootfolder,level2,level3,level4,Compound Document,Compound Document_ZTAB,Compound Document_CDMS,Compound Document_JDE,Compound Document_MAX,Compound Email,...,Url,Url_ZTAB,Url_CDMS,Url_JDE,Url_MAX,Physical Item,Physical Item_ZTAB,Physical Item_CDMS,Physical Item_JDE,Physical Item_MAX
0,Upstream Operations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Upstream Operations,Canadian Plains_NOMIGRATE,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Upstream Operations,Canadian Plains_NOMIGRATE,Applications,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Upstream Operations,Canadian Plains_NOMIGRATE,Applications,Cenovus Application Support and Maintenance,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Upstream Operations,Canadian Plains_NOMIGRATE,Applications,Request to Abandon Approval,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,Upstream Operations,Upstream Business Services,Weyburn Joint Venture,2008,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
642,Upstream Operations,Upstream Business Services,Weyburn Joint Venture,2009,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
643,Upstream Operations,Upstream Business Services,Weyburn Joint Venture,2010,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
644,Upstream Operations,Upstream Business Services,Weyburn Joint Venture,Application Support Related Documents,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# Consolidated Report of Folders with JDE and MAXIMO Links

for root_folder in ['JDE', 'MAX']:
    df_report[root_folder] = pd.DataFrame()
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Calculating {root_folder} report...')
    for folder in df_report.keys():
        if folder.find('_l3') == -1 and folder != 'JDE' and folder != 'MAX' and folder != 'Enterprise RM Workspace_NOMIGRATE' and folder != 'Planning & Exploration':
            print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     {folder}')
            df = df_report[folder][['rootfolder', 'level2', 'level3', 'level4', 'Document', 'Document_ZTAB', 'Document_CDMS', f'Document_{root_folder}', 'Folder', 'Folder_ZTAB', 'Folder_CDMS', f'Folder_{root_folder}']][df_report[folder][f'Document_{root_folder}'] + df_report[folder][f'Folder_{root_folder}'] > 0]
            if df_report[root_folder].empty:
                df_report[root_folder] = df
            else:
                df_report[root_folder] = pd.concat([df_report[root_folder], df])

    # Report output
    df_report[root_folder].set_index(['rootfolder', 'level2', 'level3', 'level4'], inplace=True)
    df_report[root_folder].to_excel(path + f'CVE - Data Analysis - {root_folder}.xlsx')
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Report Output = {df_report[root_folder].shape[0]} rows')

            

Sep 26 2023 06:55:19 AM: Calculating JDE report...
Sep 26 2023 06:55:19 AM:     Application Data
Sep 26 2023 06:55:19 AM:     Bulkloadertest_NOMIGRATE
Sep 26 2023 06:55:19 AM:     Business Development, Canadian Gas Marketing and Power
Sep 26 2023 06:55:19 AM:     CL Performance Test_NOMIGRATE
Sep 26 2023 06:55:19 AM:     Canadian Plains_NOMIGRATE
Sep 26 2023 06:55:19 AM:     Cenovus Communities
Sep 26 2023 06:55:19 AM:     CenovusRedirect_NOMIGRATE
Sep 26 2023 06:55:19 AM:     Content Server Event Calendar_NOMIGRATE
Sep 26 2023 06:55:19 AM:     Conventional Oil & Natural Gas_NOMIGRATE
Sep 26 2023 06:55:19 AM:     Corporate
Sep 26 2023 06:55:19 AM:     Corporate Development
Sep 26 2023 06:55:19 AM:     Corporate Finance
Sep 26 2023 06:55:19 AM:     Corporate Relations
Sep 26 2023 06:55:19 AM:     Corporate Services
Sep 26 2023 06:55:19 AM:     Deep Basin
Sep 26 2023 06:55:19 AM:     Development & Operations Services
Sep 26 2023 06:55:19 AM:     Down Stream
Sep 26 2023 06:55:19 AM:     E

### 3.3 Upload Category Data and create Report

In [ ]:
# Create list of extract Folders: Root Folders
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loading CVE root folders...')
cnxn = ora_engine.connect()
df_extract['cve_extract_folders'] = pd.read_sql(query['cve_root_folders'], cnxn)
cnxn.close()
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Loaded cve_root_folders = {df_extract["cve_extract_folders"].shape[0]} rows')


In [ ]:
# Create list of extract Folders: Specific Folder list
cve_folders = {'dataid': [214361471, 67272049, 1221831, 257997],
               'name': ['Deep Basin Open Invoice_Migrated to CDMS and redirecting', 'Financial Images_Migrated to CDMS & redirecting',
                      'Corporate Finance_NOMIGRATE', 'JDE Financial PDFs and Reports_TO BE ARCHIVED'],
               'path': ['Enterprise:Application Data:', 'Enterprise:Application Data:',
                      'Enterprise:Corporate Finance:Comptrollers:', 'Enterprise:Corporate Finance:']
              }
df_extract['cve_extract_folders'] = pd.DataFrame(cve_folders)

In [6]:
# Create list of extract Folders: Specific Folder list
cve_folders = {'dataid': [67272049, 1221831, 257997, 1234487, 171378752],
               'name': ['Financial Images_Migrated to CDMS & redirecting', 'Corporate Finance_NOMIGRATE',
                        'JDE Financial PDFs and Reports_TO BE ARCHIVED', 'Financial Images',
                        'JDE Attachments'],
               'path': ['Enterprise:Application Data:', 'Enterprise:Corporate Finance:Comptrollers:',
                        'Enterprise:Corporate Finance:', 'Enterprise:Upstream Operations:Finance:Central Services_NOMIGRATE:',
                        'Enterprise:Upstream Operations:Upstream Business Services:Supply Management:']
              }
df_extract['cve_extract_folders'] = pd.DataFrame(cve_folders)

In [7]:
df_report['categories'] = pd.DataFrame()
#df = df_extract['cve_root_folders'][(df_extract['cve_root_folders'].index == 31)]
#df = df_extract['cve_root_folders'] ## takes 3 hrs 20 min for all root folders
df = df_extract['cve_extract_folders']

# Build Data Assessment Report
# Extract Data
for idx, row in df.iterrows():
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: ********** FOLDER = {row["name"]} **********')
    # Upload container and contents for root folder from Database
    upload_cve_cats(row['dataid'])
    df_extract['cve_categories'] = df_extract['cve_categories'].rename(columns={'valstr': 'Category', 'items': row["name"]})

    # Create Category Matrix
    if df_report['categories'].empty:
        df_report['categories'] = df_extract['cve_categories']
    else:
        df_report['categories'] = df_report['categories'].merge(df_extract['cve_categories'], how='outer', on='Category')
df_report['categories'] = df_report['categories'].transpose(copy=False)
df_report['categories'].to_excel(path + f'CVE - Data Analysis - Categories.xlsx')

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')    

Oct 13 2023 14:25:36 PM: ********** FOLDER = Financial Images_Migrated to CDMS & redirecting **********
Oct 13 2023 14:25:36 PM: Loading CVE Folder and Document data...
Oct 13 2023 14:46:33 PM:     Loaded categories = 13 rows
Oct 13 2023 14:46:33 PM: ********** FOLDER = Corporate Finance_NOMIGRATE **********
Oct 13 2023 14:46:33 PM: Loading CVE Folder and Document data...
Oct 13 2023 15:00:38 PM:     Loaded categories = 9 rows
Oct 13 2023 15:00:38 PM: ********** FOLDER = JDE Financial PDFs and Reports_TO BE ARCHIVED **********
Oct 13 2023 15:00:38 PM: Loading CVE Folder and Document data...
Oct 13 2023 15:14:53 PM:     Loaded categories = 6 rows
Oct 13 2023 15:14:53 PM: ********** FOLDER = Financial Images **********
Oct 13 2023 15:14:53 PM: Loading CVE Folder and Document data...
Oct 13 2023 16:57:31 PM:     Loaded categories = 12 rows
Oct 13 2023 16:57:31 PM: ********** FOLDER = JDE Attachments **********
Oct 13 2023 16:57:31 PM: Loading CVE Folder and Document data...
Oct 13 2023 17

# 4. Object Importer Files

In [ ]:
# Upload Root Folders from Database
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Loading CVE root folders...')
cnxn = ora.connect(cnxn_str['cve'])
df_extract['cve_root_folders'] = pd.read_sql(query['cve_root_folders'], cnxn)
cnxn.close()
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}:     Loaded cve_root_folders = {df_extract["cve_root_folders"].shape[0]} rows')

# Create OI Files
for idx, row in df_extract['cve_root_folders'][df_extract['cve_root_folders']['DATAID'] == 213026382].iterrows():
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: ********** ROOT FOLDER = {row["NAME"]} **********')
    # Upload container and contents for root folder from Database
#    upload_cve_data(row['DATAID'])
    upload_cve_cats(row['DATAID'])

# ----- SAMPLE CODE -----

#### 2.1 Apply Mapping Tables

In [ ]:
# 2.1.1 Uploads Mapping Tables
print(f'{dt.now()}: Loading mapping tables ...')
xls = pd.ExcelFile(path + mapping_file)
mapping_tables = {}
for sheet in xls.sheet_names:
    mapping_tables['df_' + sheet] = pd.read_excel(path + mapping_file, sheet_name=sheet)

# 2.1.2 Apply Mapping Tables to retrieve FOLDER_NAME
if 'FOLDER_NAME' in df_transform.columns: df_transform.drop(['FOLDER_NAME'], axis=1, inplace=True)
if 'TYPE' in df_transform.columns: df_transform.drop(['TYPE'], axis=1, inplace=True)
if 'FOLDER_NAME_FILE' in df_transform.columns: df_transform.drop(['FOLDER_NAME_FILE'], axis=1, inplace=True)

df_transform = df_transform.merge(mapping_tables['df_project_name_from_file'][['FILE_ID', 'FOLDER_NAME']], how='left', on='FILE_ID')
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: FOLDER_NAME by File... {df_transform.shape[0]} rows affected')

df_transform = df_transform.merge(mapping_tables['df_project_name_from_prefix'], how='left', on='DOCUMENT_NUMBER', suffixes=('_FILE', None ))
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: FOLDER_NAME by Prefix... {df_transform.shape[0]} rows affected')


print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

#### 2.2 Apply Transformation

In [ ]:
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Applying transformation to Docs... {df_transform.shape[0]} rows affected')

# 2.2.1 C_REVISION
df_transform['C_REVISION'] = df_transform['REVISION'].apply(lambda x: x if pd.notnull(x) and str(x).strip() != '' else '00')
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: C_REVISION... {df_transform.shape[0]} rows affected')

# 2.2.2 C_CREATE_DATE
df_transform['C_CREATE_DATE'] = df_transform['CREATED_DATE'].apply(lambda x: '' if pd.isnull(x) 
                                                                   else (str(x)[0:10].replace('-', '') if str(x) < '2024-01-01' 
                                                                         else '19' + str(x)[2:10].replace('-', '')))
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: C_CREATE_DATE... {df_transform.shape[0]} rows affected')

# 2.2.3 C_DATE_OBSOLETE
df_transform['C_DATE_OBSOLETE'] = df_transform['DATE_OBSOLETE'].apply(lambda x: '' if pd.isnull(x) 
                                                                   else (str(x)[0:10].replace('-', '') if str(x) < '2024-01-01' 
                                                                         else '19' + str(x)[2:10].replace('-', '')))
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: C_DATE_OBSOLETE... {df_transform.shape[0]} rows affected')

# 2.2.4 C_DATE_EFFECTIVE
df_transform['C_DATE_EFFECTIVE'] = df_transform['DATE_EFFECTIVE'].apply(lambda x: '' if pd.isnull(x) 
                                                                   else (str(x)[0:10].replace('-', '') if str(x) < '2024-01-01' 
                                                                         else '19' + str(x)[2:10].replace('-', '')))
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: C_DATE_EFFECTIVE... {df_transform.shape[0]} rows affected')

# 2.2.5 C_REMARKS
df_transform['C_REMARKS'] = df_transform['REMARKS'].str.replace('\r\n', ' ')
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: C_REMARKS... {df_transform.shape[0]} rows affected')

# 2.2.6 C_SYNOPSIS
df_transform['C_SYNOPSIS'] = df_transform['SYNOPSIS'].str.replace('\r\n', ' ')
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: C_SYNOPSIS... {df_transform.shape[0]} rows affected')

# 2.2.7 BATCH
batch_size = 5000
df_batches = df_transform[['DOCUMENT_NUMBER']].drop_duplicates().reset_index(drop=True)
df_batches['BATCH'] = pd.Series(df_batches.index.values).apply(lambda x: int(x/batch_size+1))
if 'BATCH' in df_transform.columns: df_transform.drop(['BATCH'], axis=1, inplace=True)
df_transform = df_transform.merge(df_batches, how='left', on='DOCUMENT_NUMBER')
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: BATCH... {df_transform.shape[0]} rows affected')

# 2.2.8 STATUS_LAST, TITLE_LAST - Apply status and title from latest revision to all revisions
# To filter out all revisions of Deleted Documents, and avoit duplicates due to title
df_last_props = df_transform[['DOCUMENT_NUMBER', 'DOCUMENT_ID']].drop_duplicates().groupby(by='DOCUMENT_NUMBER').max().reset_index()
df_last_props = df_last_props.merge(df_transform[['DOCUMENT_NUMBER', 'DOCUMENT_ID', 'STATUS', 'TITLE']].drop_duplicates(),
                                      how='left', on=['DOCUMENT_NUMBER', 'DOCUMENT_ID'])
if 'STATUS_LAST' in df_transform.columns: df_transform.drop(['STATUS_LAST'], axis=1, inplace=True)
if 'TITLE_LAST' in df_transform.columns: df_transform.drop(['TITLE_LAST'], axis=1, inplace=True)
df_transform = df_transform.merge(df_last_props, how='left', on=['DOCUMENT_NUMBER'], suffixes=(None, '_LAST'))
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: STATUS_LAST, TITLE_LAST... {df_transform.shape[0]} rows affected')


# 2.2.9 C_FOLDER_TYPE
df_transform['C_FOLDER_TYPE'] = df_transform[['STATUS_LAST', 'TYPE']].apply(lambda x: 'DELETED' if x['STATUS_LAST'] == 'Deleted' or x['TYPE'] == 'DELETED'
                                                                                              else x['TYPE'],axis=1)

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: C_FOLDER_TYPE... {df_transform.shape[0]} rows affected')

# 2.2.10 C_FOLDER_PATH
df_transform['C_FOLDER_PATH'] = df_transform[['C_FOLDER_TYPE', 'FOLDER_NAME', 'FOLDER_NAME_FILE', 'DOCUMENT_NUMBER']].apply(lambda x: '' if x['C_FOLDER_TYPE'] == 'DELETED' or x['C_FOLDER_TYPE'] == 'DRAWING'
                                                                                                                            else (root_folder + 'UNKNOWN:' + x['DOCUMENT_NUMBER'] if x['C_FOLDER_TYPE'] == 'UNKNOWN'
                                                                                                                                  else (root_folder + x['FOLDER_NAME'] if x['C_FOLDER_TYPE'] != 'Y' and pd.notnull(x['FOLDER_NAME'])
                                                                                                                                        else (root_folder + 'PROJECTS:' + x['DOCUMENT_NUMBER'] if pd.isnull(x['FOLDER_NAME'])
                                                                                                                                              else (root_folder + 'PROJECTS:' + x['FOLDER_NAME'] if x['FOLDER_NAME'] != 'TAKE_FROM_FILE'
                                                                                                                                                    else root_folder + 'PROJECTS:' + x['FOLDER_NAME_FILE']
                                                                                                                                                   )))), axis=1)
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: C_FOLDER_PATH... {df_transform.shape[0]} rows affected')

# 2.2.11 C_DOCUMENT_ID
df_transform['C_DOCUMENT_ID'] = df_transform['C_FOLDER_PATH'] + '_' + df_transform['FILE_NAME']
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: C_DOCUMENT_ID... {df_transform.shape[0]} rows affected')

# 2.2.12 C_VERSION_NAME
df_transform['C_VERSION_NAME'] =  df_transform['FILE_ID'].astype(str) + '(rev' + df_transform['C_REVISION'].astype(str) + ') ' + df_transform['FILE_NAME']
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: C_VERSION_NAME... {df_transform.shape[0]} rows affected')

# 2.2.13 C_EBWEB_FILE_PATH
df_transform['C_EBWEB_FILE_PATH'] = df_transform['REPOSITORY'] + df_transform['PATH']
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: C_EBWEB_FILE_PATH... {df_transform.shape[0]} rows affected')

# 2.2.14 C_EXTRACT_FILE_PATH
df_transform['C_EXTRACT_FILE_PATH'] = df_transform['C_EBWEB_FILE_PATH'].apply(lambda x: x.replace('E:', '\\\wsidm009pd'))
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: C_EXTRACT_FILE_PATH... {df_transform.shape[0]} rows affected')

# 2.2.15 C_UPLOAD_FILE_PATH
df_transform['C_UPLOAD_FILE_PATH'] = upload_path + df_transform['C_VERSION_NAME']
#df_transform['C_UPLOAD_FILE_PATH'] = upload_path + df_transform['FILE_NAME']
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: C_UPLOAD_FILE_PATH... {df_transform.shape[0]} rows affected')

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')


## Step 3. Data Assessment

#### 3.0 Create Assessment Functions

In [ ]:
# 3.0.1 Raw Column Counts and Single Column Lookups
def column_assessment(df:pd.DataFrame, col_lookup_skip:list):
    df_col_counts = pd.DataFrame()
    col_len = {}
    col_newline = {}
    col_lookup_vals = {}

    for col in df.columns:
        count_newline = 0
        col_len[col] = int(df[col].astype(str).str.len().max())
        col_newline[col] = int(df[col].astype('str').str.find('\r\n').apply(lambda x: 1 if x > 0 else 0).sum())
        if col not in col_lookup_skip:
            col_lookup_vals[col] = df[[col, 'DOCUMENT_ID']].groupby(by=col).count()

    df_col_counts = pd.DataFrame(df.count()).rename(columns={0: 'Count All'})
    df_col_counts = df_col_counts.merge(pd.DataFrame(df.nunique()).rename(columns={0: 'Count Unique'})
                                        ,how='outer', right_index=True, left_index=True)
    df_col_counts = df_col_counts.merge(pd.DataFrame(pd.Series(col_len), columns=['Max Length'])
                                        ,how='outer', right_index=True, left_index=True)
    df_col_counts = df_col_counts.merge(pd.DataFrame(pd.Series(col_newline), columns=['With New Line char'])
                                        ,how='outer', right_index=True, left_index=True)
    return df_col_counts, col_lookup_vals

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')


#### 3.1 Column Data Assessment

In [ ]:
# 3.1.1 Column Data Assessment
lookup_skip_columns = ['DOCUMENT_ID', 'DOCUMENT_NUMBER', 'PROJECT_CODE', 'TITLE']
df_col_counts, col_lookup_vals = column_assessment(df_extract , lookup_skip_columns) 

with pd.ExcelWriter(path + 'Lima - Project - Columns Data Assessment.xlsx') as writer:
    df_col_counts.to_excel(writer, sheet_name='Counts')
    for col in col_lookup_vals:
        if not col_lookup_vals[col].empty: col_lookup_vals[col].to_excel(writer, sheet_name = 'Values ' + col)

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

#### 3.3 Validation & Counts

In [ ]:
# 3.3.1.a Duplicated File Ids in EXTRACT
# (cartesian product: Caused by having same file associated to 2 different copies in eBWeb)
df_dup_file_id_copy = df_extract[['FILE_ID', 'DOCUMENT_ID']].groupby(by='FILE_ID').count()
df_dup_file_id_copy = df_dup_file_id_copy[(df_dup_file_id_copy['DOCUMENT_ID'] > 1)]
print(f'Same File Ids in multiple copies: \n {df_dup_file_id_copy}')

# 3.3.1.b Duplicated File Ids in TRASNFORM
# (NOT an error: Copied same files to multiple target folders per user request)
df_dup_file_id_folder = df_transform[['FILE_ID', 'DOCUMENT_ID']].groupby(by='FILE_ID').count()
df_dup_file_id_folder = df_dup_file_id_folder[(df_dup_file_id_folder['DOCUMENT_ID'] > 1)]
print(f'\n Same File Ids in multiple target folders: \n {df_dup_file_id_folder}')

# 3.3.2 Duplicated Document Names in same Folder
df_dup_docs = df_transform[['C_FOLDER_PATH', 'FILE_NAME', 'TYPE']][(df_transform['C_FOLDER_TYPE'] != 'DELETED')].drop_duplicates().groupby(by=['C_FOLDER_PATH', 'FILE_NAME']).count()
df_dup_docs = df_dup_docs[df_dup_docs['TYPE']>1]
print(f'\n Duplicated File Names within Target Folders: \n {df_dup_docs}')

# 3.3.3 Duplicated Filenames in each Revision
#(catesian product: created by dup DOCUMENT_NUMBER in the mapping table. 
#Dups are caused by the same PREFIX having different TITLE in each revisions)
df_dup_revs = df_transform[['C_FOLDER_PATH', 'C_VERSION_NAME', 'C_REVISION', 'TYPE']][(df_transform['C_FOLDER_TYPE'] != 'DELETED')].groupby(by=['C_FOLDER_PATH', 'C_VERSION_NAME', 'C_REVISION']).count()
df_dup_revs = df_dup_revs[df_dup_revs['TYPE']>1]
print(f'\n Document Numbers with diff Title in multiple revisions causing filenames cartesian products: \n {df_dup_revs}')

# 3.3.4 Documents with missing mapping
df_missing_mapping = df_transform[['PROJECT_CODE', 'TITLE', 'DOCUMENT_NUMBER', 'STATUS']][(df_transform['C_FOLDER_TYPE'].isnull())].drop_duplicates()
print(f'\n Document Numbers without a type/folder mapping: \n {df_missing_mapping}')

# 3.3.5 Documents with multiple versions
df_mult_vers = df_transform[['C_FOLDER_PATH', 'FILE_NAME', 'FILE_ID']][(df_transform['C_FOLDER_TYPE'] != 'DELETED')].groupby(by=['C_FOLDER_PATH', 'FILE_NAME']).count()
df_mult_vers = df_mult_vers[df_mult_vers['FILE_ID']>1]

# 3.3.6 Counts per FOLDER TYPE
df_count_per_folder = df_transform[['C_FOLDER_TYPE', 'C_FOLDER_PATH', 'DOCUMENT_ID', 'C_DOCUMENT_ID', 'FILE_ID', 'FILE_SIZE']].groupby(by=['C_FOLDER_TYPE'], dropna=False).agg({'C_FOLDER_PATH': pd.Series.nunique, 'DOCUMENT_ID': pd.Series.nunique, 'C_DOCUMENT_ID': pd.Series.nunique, 'FILE_ID': pd.Series.count, 'FILE_SIZE': lambda x: np.sum(x)/1024/1024/1024})
df_count_per_folder.rename(columns={'C_FOLDER_PATH': 'Folders', 'DOCUMENT_ID': 'Folders_Revisions', 'C_DOCUMENT_ID': 'Documents', 'FILE_ID': 'Versions'}, inplace=True)

# 3.3.7 Counts per FOLDER TYPE and BATCH
df_count_per_batch = df_transform[['C_FOLDER_TYPE', 'BATCH', 'C_FOLDER_PATH', 'C_DOCUMENT_ID', 'FILE_ID', 'FILE_SIZE']].groupby(by=['C_FOLDER_TYPE', 'BATCH'], dropna=False).agg({'C_FOLDER_PATH': pd.Series.nunique, 'C_DOCUMENT_ID': pd.Series.nunique, 'FILE_ID': pd.Series.count, 'FILE_SIZE': lambda x: np.sum(x)/1024/1024/1024})
df_count_per_batch.rename(columns={'C_FOLDER_PATH': 'Folders', 'C_DOCUMENT_ID': 'Documents', 'FILE_ID': 'Versions'}, inplace=True)

with pd.ExcelWriter(path + 'Lima - Project - Counts.xlsx') as writer:
    df_dup_file_id_copy.to_excel(writer, sheet_name='Dup Files in many copies')
    df_dup_file_id_folder.to_excel(writer, sheet_name='Dup Files in many folders')
    df_dup_docs.to_excel(writer, sheet_name='Dup Documents in same folder')
    df_dup_revs.to_excel(writer, sheet_name='Dup Files in same revision')
    df_missing_mapping.to_excel(writer, sheet_name='Missing Mapping')
    df_mult_vers.to_excel(writer, sheet_name='Multiple Versions')
    df_count_per_folder.to_excel(writer, sheet_name='Counts per Folder')
    df_count_per_batch.to_excel(writer, sheet_name='Counts per Batch')

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

## Step 4. Load Layer

#### 4.1 XML DataFrames

In [ ]:
migration_date = dt.now().strftime("%Y%m%d%H%M%S")

# 4.1.1 Create XML Documents DataFrame
for ft in df_transform['C_FOLDER_TYPE'].unique():
    xml = {}
    df = df_transform[(df_transform['C_FOLDER_TYPE'] == ft)].sort_values(by=['C_FOLDER_PATH', 'FILE_NAME'])
    nodes = len(df)
    xml = {'title': df['FILE_NAME'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'location': df['C_FOLDER_PATH'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'description': df['TITLE_LAST'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'created': df['C_CREATE_DATE'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'createdby': ['Admin'] * nodes,
           'modified': [migration_date] * nodes,
           # Version
           'filename': df['C_VERSION_NAME'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'vercdate': df['C_CREATE_DATE'].apply(lambda x: str(x) if pd.notnull(x) else 0),
           'file': df['C_UPLOAD_FILE_PATH'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'mime': df['MIME_TYPE'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           # Id's
           'unique_doc_id': df['C_DOCUMENT_ID'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'batch': df['BATCH'],
           'ebweb_path': df['C_EBWEB_FILE_PATH'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'extract_path': df['C_EXTRACT_FILE_PATH'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           # Categories
           'Number': df['DOCUMENT_NUMBER'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Title': df['TITLE'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Revision': df['REVISION'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Units': df['UNITS'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Document Type': df['DOCUMENT_TYPE'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Document Classification': df['DOCUMENT_CLASSIFICATION'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Status': df['STATUS'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Document Status': df['DOCUMENT_STATUS'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Date Obsolete': df['C_DATE_OBSOLETE'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Date Effective': df['C_DATE_EFFECTIVE'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Paper Destroyed': df['PAPER_DESTROYED'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Vendor Name': df['VENDOR_NAME'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Vendor Doc Number': df['VENDOR_DOC_NUMBER'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Remarks': df['C_REMARKS'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Synopsis': df['C_SYNOPSIS'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Project Codes': df['PROJECT_CODE'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Document Id': df['DOCUMENT_ID'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'File Id': df['FILE_ID'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Repository': df['REPOSITORY'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'Path': df['PATH'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'File Name': df['FILE_NAME'].apply(lambda x: str(x).strip() if pd.notnull(x) else ''),
           'File Size': df['FILE_SIZE'].apply(lambda x: str(x).strip() if pd.notnull(x) else '')          
          }
    df_xml[ft] = pd.DataFrame(xml)
    print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Documents {ft} = {df_xml[ft].shape[0]} rows')
   
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')


#### 4.2 Copy Files

In [ ]:
# 4.2.1 Copy batch Files.
#extract_path = 'E:\\eBWeb Test Export\\Projects\\'
for ft in df_xml:
    df = df_xml[ft]
    batches = df['batch'][(df['batch'].notnull())].sort_values().unique()
    for batch in batches:
        df_batch = df[df['batch'] == batch]
        bat_file_name = 'copy_projects-' + ft + '-' + str(batch).zfill(2)
        bat_file = open(path + xml_path + bat_file_name  + '.bat', 'w',  encoding='utf-8')
        cmd = f'ECHO start %time% >> "{upload_path + bat_file_name}.log"'
        bat_file.write(cmd + '\n')    
        for key, val in df_batch.iterrows():
            cmd = f'ECHO "{val["filename"]}" >> "{upload_path + bat_file_name}.log"'
            bat_file.write(cmd + '\n')
            cmd = f'IF EXIST "{val["extract_path"]}" (ECHO F | XCOPY /Y /Q /F "{val["extract_path"]}" "{val["file"]}" >> "{upload_path + bat_file_name}.log") ELSE (ECHO "{val["ebweb_path"]}" does not exist) >> "{upload_path + bat_file_name}.log"'
            bat_file.write(cmd + '\n')
        cmd = f'ECHO end %time% >> "{upload_path + bat_file_name}.log"'
        bat_file.write(cmd + '\n')    
        bat_file.close()
        print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: {bat_file_name} = {df_batch.shape[0]}')

print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')

#### 3.3 Robocopy Files

In [ ]:
# 3.3.1 Robocopy: Documents, Emails, Shortcuts
"""df = pd.DataFrame()

bat_file = open(xml_output_path + 'bat_file.txt', 'w',  encoding='utf-8')
            
for pf in df_xml:
    for type in ['document', 'email']:
        df = df_xml[pf][(df_xml[pf]['type'] == type)]
        batches = df['batch'][(df['batch'].notnull())].sort_values().unique()
        for batch in batches:
            copy_folder = pf.replace(' ', '_') + '-' + str(batch).zfill(2)
            robocopy_cmd =  'robocopy \\\cidvsrv08.huskyenergy.ca\\Divest_export\\WellFiles\\' + copy_folder +  '\\ ' + upload_path + copy_folder + '\\ /r:2 /MT:16 /log:' + upload_path + copy_folder + '.log' 
#            print(robocopy_cmd)
            bat_file.write(robocopy_cmd + '\n')
bat_file.close()
print(f'{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Done')
"""
#robocopy "\\LIREB1\eBWeb Test Export2\Equipment" \\cidvsrv08\Divest_export\eBWeb\Equipment /r:2 /log:\\cidvsrv08\Divest_export\eBWeb\Equipment.log

#### 3.4 Object Importer Files

In [ ]:
# 3.4.1 Create DOCUMENTS
df = pd.DataFrame()
for ft in ['Y', 'N - SPEC', 'N - REFERENCE', 'N - BORE', 'N - PROCEDURE', 'UNKNOWN']:
    df = df_xml[ft]
    if not df.empty:
        print(f'\n{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Processing {ft} = {df.shape[0]} rows')
        build_create_oi(df = df,
                        type = 'document',
                        output_file = ft,
                        sort_list = ['location', 'title'],
                        legacy_id = 'unique_doc_id',
                        category_name = 'Content Server Categories:OPR-REF-Lima:OPR-REF-Lima-Legacy Projects',
                        cat_atts = ['Number', 'Revision', 'Title', 'Units', 'Document Type', 'Document Classification', 'Status', 'Document Status',
                                    'Date Obsolete', 'Date Effective', 'Paper Destroyed', 'Vendor Name', 'Vendor Doc Number',
                                    'Remarks', 'Synopsis', 'Project Codes', 'File Id']
                       )
    

In [ ]:
# 3.4.3 Delete DOCUMENTS
for ft in  ['Y', 'N - SPEC', 'N - REFERENCE', 'N - BORE', 'N - PROCEDURE', 'UNKNOWN']:
    df = df_xml[ft][['location', 'title', 'batch']].drop_duplicates()
    if not df.empty:
        print(f'\n{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Processing {ft} = {df.shape[0]} rows')
        build_delete_oi(df = df,
                        type = 'document',
                        output_file = ft,
                        sort_list = ['location', 'title']
                       )
print('Done')

In [ ]:
# 3.4.3 Update DOCUMENTS to add Legacy eB Web category
df = pd.DataFrame()
for ft in ['Y', 'N - SPEC', 'N - REFERENCE', 'N - BORE', 'N - PROCEDURE', 'UNKNOWN']:
    df = df_xml[ft]
    if not df.empty:
        print(f'\n{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Processing {ft} = {df.shape[0]} rows')
        build_update_oi(df = df,
                        type = 'document',
                        output_file = ft,
                        sort_list = ['location', 'title'],
                        category_name = 'Content Server Categories:Legacy eB Web',
                        cat_atts = ['Document Id', 'File Id', 'Repository', 'Path', 'File Name', 'File Size']
                       )

In [ ]:
# 3.4.4 FIX UNITS: Re-apply value after updating Category
df = pd.DataFrame()
for ft in ['Y', 'N - SPEC', 'N - REFERENCE', 'N - BORE', 'N - PROCEDURE']:
    df = df_xml[ft]
    if not df.empty:
        print(f'\n{dt.now().strftime("%b %d %Y %H:%M:%S %p")}: Processing {ft} = {df.shape[0]} rows')
        build_update_oi(df = df,
                        type = 'document',
                        output_file = f'fix-{ft}',
                        sort_list = ['location', 'title'],
                        category_name = 'Content Server Categories:OPR-REF-Lima:OPR-REF-Lima-Legacy Projects',
                        cat_atts = ['Units']
                       )